# Download YouTube Video's Audio

In [39]:
! pip install pytube -q

In [2]:
from pytube import YouTube

In [3]:
#VIDEO_URL = "https://www.youtube.com/watch?v=hWLf6JFbZoo" #obama

In [4]:
VIDEO_URL ="https://www.youtube.com/watch?v=TzPS6ELoZ1I" #english ad 30 sec about sugary juices

In [5]:
#VIDEO_URL = 'https://www.youtube.com/watch?v=h-JVjs9AAmQ' #batman

In [ ]:
#VIDEO_URL = 'https://youtu.be/qNJRGHk7sN8'

In [6]:
yt = YouTube(VIDEO_URL)

In [7]:
yt.streams.filter(only_audio=True, file_extension='mp4').first().download(filename='ytaudio.mp4')


'/content/ytaudio.mp4'

In [8]:
! ffmpeg -i ytaudio.mp4 -acodec pcm_s16le -ar 16000 ytaudio.wav

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [9]:
!pip install torch==2.0.1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# English ASR with HuggingSound

In [10]:
!pip install huggingsound -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 97.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 91.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [11]:
from huggingsound import SpeechRecognitionModel


In [12]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [13]:
device

'cpu'

In [14]:
model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-english", device = device)


INFO:huggingsound.speech_recognition.model:Loading model...


OUT OF MEMORY (OOM) error

# Audio Chunking

In [15]:
import librosa

In [16]:
input_file = '/content/ytaudio.wav'

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
print(librosa.get_samplerate(input_file))

# Stream over 30 seconds chunks rather than load the full file
stream = librosa.stream(
    input_file,
    block_length=30,
    frame_length=16000,
    hop_length=16000
)

16000


In [19]:
import soundfile as sf


In [20]:
for i,speech in enumerate(stream):
  sf.write(f'{i}.wav', speech, 16000)

In [21]:
i

1

# Audio Transcription / ASR / Speech to Text

In [22]:
audio_path =[]
for a in range(i+1):
  audio_path.append(f'/content/{a}.wav')

In [23]:
audio_path

['/content/0.wav', '/content/1.wav']

In [24]:
transcriptions = model.transcribe(audio_path)

100%|██████████| 2/2 [00:51<00:00, 26.00s/it]


In [25]:
full_transcript = ' '

In [26]:
for item in transcriptions:
  full_transcript += ''.join(item['transcription'])

In [41]:
len(full_transcript)

221

# Text Summarization

In [28]:
from transformers import pipeline

In [45]:
from transformers import pipeline

model_name = "sshleifer/distilbart-cnn-12-6"  # Example model, replace with your desired model

summarization = pipeline('summarization', model=model_name)

full_transcript = full_transcript # Your input text for summarization
max_length = 23  # Adjust the max_length value according to your needs
min_length = 10  # Adjust the min_length value according to your needs

summary = summarization(full_transcript, max_length=max_length, min_length=min_length)

print(summary[0]['summary_text'])


Your max_length is set to 23, but your input_length is only 3. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)


 CNN.com will feature iReporter photos in a weekly Travel Snapshots gallery . Please submit your


In [46]:
summarized_text = summarization(full_transcript)

Your max_length is set to 142, but your input_length is only 3. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)


In [31]:
summarized_text[0]['summary_text']

' juice drinks sodas and sports drinks are loaded with added sugars and caleries and have little or no nutritional values . Sugary drinks can destroy teeth causovesity and lead to type two diabetessugar bites protect your kids from being bitten by sugary drinks and bites .'

Text Chunking before Summarization

In [42]:
num_iters = int(len(full_transcript)/1000)
summarized_text = []
for i in range(0, num_iters + 1):
  start = 0
  start = i * 1000
  end = (i + 1) * 1000
  #print("input text \n" + full_transcript[start:end])
  out = summarization(full_transcript[start:end], min_length = 5, max_length=20)
  out = out[0]
  out = out['summary_text']
  print("Summarized text\n"+out)
  summarized_text.append(out)

print(summarized_text)

Summarized text
 juice drinks sodas and sports drinks are loaded with added sugars and caleries and have
[' juice drinks sodas and sports drinks are loaded with added sugars and caleries and have']
